In [2]:
import torch
import numpy as np
from PIL import Image
from torchvision.transforms.functional import to_pil_image
from facenet_pytorch.models.mtcnn import PNet, RNet, ONet  # Import P-Net, R-Net, O-Net
from facenet_pytorch import MTCNN
import os
import matplotlib.pyplot as plt
from facenet_pytorch import InceptionResnetV1
import cv2
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn.functional import normalize
import pyodbc as odbc
import json
from datetime import datetime


c:\Users\harsh.n\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [126]:
SERVER_NAME = '192.168.1.101'
DRIVER_NAME = 'ODBC Driver 17 for SQL Server'
DATABASE_NAME = 'AI_Face_Recognition'
cnxn = odbc.connect('DRIVER={ODBC Driver 17 for SQL Server}; \
                    SERVER='+SERVER_NAME+'; \
                    DATABASE = '+DATABASE_NAME+'; \
                    Uid=sa;Pwd=Masterly@123;')
cur = cnxn.cursor()

In [127]:
cur.execute("use AI_Face_Recognition")

In [129]:
Studio_image_path = r'E:\2024\hires\324\Studio\20240222\2024-JUNE 13-TEAM-38753-20240222-LT OUTLAWS-ROSTER-467624-P-DECLAN KOLLUS (P)-1.jpg'

In [130]:
# Define fine-tuned P-Net, R-Net, and O-Net for finetuning
class FinetunedMTCNN(MTCNN):
#(self, image_size=160, margin=5, **kwargs): # use this after some time to improve the final results
    def __init__(self, **kwargs):
        super(FinetunedMTCNN, self).__init__(**kwargs)
        # Create your custom, finetuned P-Net, R-Net, O-Net here
        self.pnet = PNet()
        self.rnet = RNet()
        self.onet = ONet()
    def forward(self, x):
        # Overriding forward pass if additional finetuning is needed
        return super().forward(x)

In [131]:
# Initialize FaceNet model
facenet = InceptionResnetV1(pretrained='vggface2').eval()  # Load the FaceNet model
# Initialize mtcnn model
finetuned_mtcnn = FinetunedMTCNN(keep_all=True, device='cuda:0' if torch.cuda.is_available() else 'cpu', min_face_size=60)

In [132]:
def find_euclidean_distance(src, dst):
    return np.linalg.norm(src - dst)

In [133]:
def alignment_procedure(img, left_eye, right_eye):    
    left_eye_x, left_eye_y = left_eye
    right_eye_x, right_eye_y = right_eye
    
    # Find the direction to rotate the image based on the eye coordinates
    if left_eye_y > right_eye_y:
        point_3rd = (right_eye_x, left_eye_y)
        direction = -1  # Clockwise
    else:
        point_3rd = (left_eye_x, right_eye_y)
        direction = 1  # Counter-clockwise
    
    # Calculate the length of the triangle edges
    a = find_euclidean_distance(np.array(left_eye), np.array(point_3rd))
    b = find_euclidean_distance(np.array(right_eye), np.array(point_3rd))
    c = find_euclidean_distance(np.array(left_eye), np.array(right_eye))
    
    # Apply cosine rule to find the angle
    if b != 0 and c != 0:  # Avoid division by zero
        cos_a = (b**2 + c**2 - a**2) / (2 * b * c)
        angle = np.arccos(cos_a)  # Angle in radians
        angle = np.degrees(angle)  # Convert to degrees
        
        # Adjust the angle based on the rotation direction
        if direction == -1:
            angle = 90 - angle
        
        # Rotate the image using PIL
        #img = Image.fromarray(img)
        img = img.rotate(direction * angle, resample=Image.BICUBIC)
        img = np.array(img)  # Convert back to numpy array
        # aligned_img_pil = Image.fromarray(img)  # Convert back to PIL Image for saving
        # aligned_img_pil.show()
    return img

In [134]:
# Function to perform face detection and store image paths with their cropped face regions
def get_faces_studio(mtcnn_model, margin=0):
            global studio_photos_all, studio_photos_paths
            studio_photos_all = []
            studio_photos_paths = []
            image_path = Studio_image_path
            image = Image.open(image_path)
            boxes, confidences, landmarks = mtcnn_model.detect(image, landmarks=True)
            if boxes is not None:
                # Set a confidence threshold
                threshold = 0.95
                # Filter detected faces based on the confidence score
                filtered_faces = [i for i, confidence in enumerate(confidences) if confidence > threshold]
                # Process each filtered face
                for i in filtered_faces:
                    box = boxes[i]  # Get the bounding box for the filtered face
                    box = [int(b) for b in box]  # Ensure the box is in integer format
                    # adding margin around the box
                    # Apply margin to the bounding box
                    x1 = max(0, box[0] - margin)  # Left
                    y1 = max(0, box[1] - margin)  # Top
                    x2 = min(image.width, box[2] + margin)  # Right
                    y2 = min(image.height, box[3] + margin)  # Bottom
                    # Crop the face from the image
                    cropped_face = image.crop((x1, y1, x2, y2))
                    if cropped_face is not None:
                        # Get the landmarks (left and right eyes) for the current face
                        left_eye = landmarks[i][0]  # Left eye coordinates for face i
                        right_eye = landmarks[i][1]  # Right eye coordinates for face i
                        # Align the cropped face using the eye coordinates
                        aligned_face = alignment_procedure(cropped_face, left_eye, right_eye)
                        # Store the aligned face and the original image path
                        studio_photos_all.append(aligned_face)
                        studio_photos_paths.append(image_path)  # Store original image path 
                        
            return studio_photos_paths


In [31]:
x = []
for i in x:
    print(2+2)

In [135]:
def get_face_embedding(image):
    image = cv2.resize(image, (160, 160))
    image = torch.tensor(image).permute(2, 0, 1).unsqueeze(0).float() / 255.0
    embedding = facenet(image)
    # this can be added or removed according to the data (pending decision)
   # embedding = normalize(embedding, p=2, dim=1)# L2 normalization of embeddings
    np_embedding =   embedding.detach().numpy()
    return np_embedding

In [136]:
# Process both match and studio images, and return original image paths
studio_photos_paths = get_faces_studio(finetuned_mtcnn, margin=200)

In [ ]:
# Generate embeddings for studio photos
studio_embedding = []
for studio_photo in studio_photos_all:
    studio_image = np.array(studio_photo)  # Convert the PIL image to a NumPy array
    embeddings = get_face_embedding(studio_image)  # Get embedding
    for embedding in embeddings:
        embedding_str = json.dumps(embedding.tolist())    
        studio_embedding.append(embedding)


[0.008038179948925972, 0.04973822832107544, 0.03338813781738281, -0.04738432541489601, 0.047333065420389175, 0.009302068501710892, 0.012250423431396484, -0.056327078491449356, 0.04163767397403717, 0.05838329344987869, 0.016510959714651108, 0.027795398607850075, -0.022474460303783417, 0.030728504061698914, -0.02278393879532814, -0.003561483696103096, -0.023383134976029396, 0.002546887844800949, 0.012162278406322002, 0.06040704995393753, -0.019420387223362923, -0.021191813051700592, 0.07958200573921204, -0.07489603012800217, -0.05457344278693199, 0.024988580495119095, 0.01793675683438778, -0.0014716885052621365, -0.004705463536083698, -0.008717170916497707, 0.05467298999428749, 0.0364239402115345, 0.0024558540899306536, -0.03288187086582184, 0.03807282820343971, -0.04578068107366562, -0.0002338398335268721, 0.0022584351245313883, -0.0058266595005989075, 0.016943980008363724, -0.01282658614218235, -0.056612927466630936, -0.034499846398830414, 0.10169484466314316, 0.013047096319496632, -0.

ProgrammingError: ('Invalid parameter type.  param-index=2 param-type=set', 'HY105')

In [18]:
emb = 0x0159CBBD6F85B63C9429323C2F2E11BD33396A3DE09263BC032318BBB67452BD6282423D9E20C23DDAEFB63CE915B93DF26B893BC57857BC3D63B7BDE01B463D7E57C13BC049D43C72EA93BC38A9E6BC30A5FFBC2B35433CB7F9D33D786E06BD4DA105BD097EF43B1D48E43BCA2FE6BC290D86BD95054EBCF15014BD3F711E3DEC09453B291B3F3DF48E2F3C65DDB8BCC06B6C3DA236D0BD06CC0CBE19FC883CE9C789BBA2E16BBD7788253D117CD63D9FD000BC710F023D14DA013DCEAB8C3D5AAD24BD99C687B9E4BDDFBC263AEFBC80C9523C4BCE21BDF79E783D8B3B983B66A2A2BD22638C3D9805C9BB6B0A793C7AFD7E3AEBEE0D3DEBCC02BC868AF13BB36723BBC3D25CBC8F58CEBDD336573DB6478CBCE4FF2C3D8AEF573DF10F8EBCFB7DB83D43D65F3D3911933CA454CDBB8FD52BBCF0C7E53C5381313C189A1ABD72C495BA6767A0BC385B113BCD9111BDED27A5BD6F555B3C5C5D23BDABCCB63CF5F49A3C89BE33BDC959853BA2CB793D2038363D441992BA57E9C4BCCBE9793C4AAB5D3C4CC5E43DBDD4C63C365C143DBED1FB3C77F6033D3C1143BD0D29F63C00D583BD61C006BC1E35403D2A7C43BD5553CBBCFB0B0DBC45DB5A3B39B04CBD56C9A8BB62BA363D500302BCE6257E3C1632BD3CC48C8A3DA0979A3C271F053CB550B43D5DD2D4BCFF6EA63CCDDA0EBCF502D5BB160E8ABC77999A3C1A7BA73DDE49D73C4A4F55BD496D07BDCD0C9ABDEBA8E63C3982813C4DF988BDE4BA7FBC7FE8A3BD151E62BD2DBA1FBD9B680B3DEB50F8BC5609C33A895E23BD7669423C78F8B4BB87AC03BD128B87BB8E93303C2BEBAC3DCC76F83CF9E8063C80F51F3D1DA01ABC4FBEBE3D3C41C8BC752D82BDD0AFEEBD04AE54BD52FD50BD3E0034BDBA52D53C3698803D7B4F22BD1700A23DE456233D418539BD0035863C74A2A3BBEBA21EBDB174F43BD17F7FBC5EDDEEBC6B1C98BD408B813DAC302ABDC38C1C3A66EDBB3C4C9D003EEDAE1ABD0BFA99BB1E2A29BAB1FB873D901E113D0E789D3D088ECB3C337E44BDC454D33C66FA023C70F0233DEB3A263D2B28C03DF4555F3D4058283C68F2A5BC480582BCC1357ABCA0C5B23C9B2F0B3D5AFAF73B9A44A93B5222843D6CC764BD908FDDBB315991BCD67FE43C38AD8B3B78E4963DFB4A0EBC5139E9BAE3DD06BDEA2DB33C540B013CCCB6173D4FBA523DC3D3EE3C20C1C23B40CA4B3A16D482BB65084ABD3CED6D3D490176BD45125C3CC665BF3D385E9DBD93E7C23CB4748B3DA9E2B1BC6FE4C73C349C163D8B6CAE3D83CA453B8D8110BCF506F1BC543F4A3D8FAA67BC5ACE183D66ABD8BC10345FBC2752A8BB26FE233D29E4A3BC8C20523D5FD0C83C6F65DE3CE75E1A3B7557973C3D1E1E3CACAB573D217ED83DB4A4FE3CB06E06BC79AD73BBC9824D3DA245C7BCA7645EBDC4D46B3B14F0EDBC7654293DEE4CF0BB6293013DBB0015BCC469BD3C37D846BD52F1523DBE4154BDBB1A8CBD2797B2BCA6B3D1BB2FA6373DD43E04BDC8A1883DEB38B7BC24D0B03BCAF91F3DDC30313CB87EDEBCB6B71C3DDF94AA3D7958783DA35C3E3D9D4F723D0F4E783CCE9C52BDB65B83BD51A8BB3C511D8ABDBD4321BDBC63CABB48132CBC631A87BA196E98BCB155693D1774D5BCA5D2413C4B1F073D47CAE3BC4B296F3C50A21ABCA4CBF63C88D3213D24CBCF3B77689A3D52CD17BC2F59AE3D3F85B13DE5FDD33CCE2ECD3CD59D49BDD4D450BD2E6EC83B4A9D273CBF60A43DFF48A4BCE46526BD916FC23DEBC84F3D2B2C6F3CF18D7B3DBB59B63DAF4FA2BD5120833D6CBE85BD4F758FBC82D9603D096113BDACE3293DC925E23C18BA023D8FFD663C8827B1BB35D2F63C8434AB3CDBCC3D3B4DC6553C23652D3B2C3F863D7C5C1B3D7C77F03C4CDC18BD25D818BDFF2C12BCB8FD723DE8787D3C7F6C07BDAD399A3C0E0C5E3D48488BBD7E5C9A3D2448413D89D21E3D43E81C3DA3F44EBDE18661BBE631703DA083EDBC908F3DB8C1A91D3DF083593CF6EA503D13819C3A2ECEF0BC5EE042BDE0714DBCA1E25EBD9CD7603CF97B26BDC5EE6ABD74F070BBB5D498BCFBCA07BC88DC88BCBA961F3D063A9ABDC5C8E83C4167723C0B43C03DCA8DBB3CC50CF13C0FA19BBC3FAB3C3DF661DCBCA3A4513D5105873DD76F9FBC8890463CBAB98EBA2A75F9BCA6A1B3BCD1BE80BC8F1CD4BCB06358BDC07B1C3CB44A893DB1433F3AE3E09F3D714C99BD0A808D3DF82BE53CDE0A9A3C11C09FBC05633A3D5AE50EBD41601C3D67DD8A3CCA1B433C1A1D82BDD243083DFBC6A6BBB6B2183D0A46D03D3A56E539BB7A953ACD5BBF3CD2C58A3D2C58A33C61B8D63C8E21403D97A1043DED6C93BDD3B4833C92DF6F3D1395DA3C40AD513DF23E543DFB35B23B0BAB443D209B083DC69DFA391A000CBD6EA160BD91F77EBD9400343D4683F6BC8976A9BC2FECE63CD2990F3D6425D73B947A23BCC74B05BDB2953ABC8D779BBDD1D179BC0E2EA43CB1DD113D09D0C3BA5102E0BCE75425BDB5A0DDBCE778403CD596B2BCFAE0653DD0E8873DCA98BA3C1D14843B5ED80CBDF6EB193DE05ADDBDBCF5493C6064513D8C4005BD13975E3D8C27C33C9670EEBC156671BC0C3E083D26BB8FBC52B817BDF18431BD2DE2583D5FF890BC220E233D0811243DED8CB83B5599B7BCE5153FBD6AB39F3D8414173DA0A76F3D82EC3F3D865D0A3DC8BDF9BB6173DE3C7CEA3FBD711C913D661DD63A0FED4DBD960E2FBDF390AD3C5E45BFBCB73500BE987954BCB1E5B23CE2299ABD70967BBD2B22C83D22351FBD22AD82BDB556C33D78DAF43C1C069BBCD517D5BD71BEEC3CA84D4FBCFC66FCBB4238BABC943CA43C0779D6BDC35E3DBD5933D7BB123DE1BC42A6B23C9C295EBC
print(type(emb))

<class 'int'>


In [ ]:
emb_bytes = emb.to_bytes((emb.bit_length() + 7) // 8, byteorder='big')
retrieved_array = np.frombuffer(emb_bytes, dtype=np.float32).reshape(1, -1)

In [26]:
import numpy as np

# Assuming array_1 and array_2 are NumPy arrays
are_identical = np.array_equal(retrieved_array, array_2)

print("Are the arrays identical?", are_identical)


Are the arrays identical? False


In [ ]:
print(retrieved_array)

In [124]:
from datetime import datetime

current_timestamp = datetime.now().isoformat()
print(current_timestamp)


2024-11-14T11:18:30.912824


In [115]:
cur.execute("""
INSERT INTO PlayerPhotoEmbedding (PlayerEmbeddingID, RosterID, TournamentID, SFaceembeddings, ImagePath, TeamKey, UpdatedOn, CreatedOn)
VALUES(1,1,1,1,1,1,1,1)    """)


ProgrammingError: Attempt to use a closed cursor.

In [ ]:
cur.execute("""
    INSERT INTO PlayerEmbeddings (
        RosterId,
        TeamKey,
        FaceEmbeddings,
        ImagePath,
        TournamentId,
        UpdatedOn,
        CreatedOn
    )
    SELECT 
        c.RosterId,
        c.TeamKey,
        :face_embeddings,
        :image_path,
        c.tournament_id,
        CURRENT_TIMESTAMP,
        CURRENT_TIMESTAMP
    FROM 
        Constellation c
    WHERE 
        c.RosterId = :roster_id;
""", {
    'face_embeddings': face_embeddings,
    'image_path': image_path,
})

In [ ]:
query = """INSERT INTO PlayerPhotoEmbedding (RosterID, TournamentID, SFaceembeddings, ImagePath, TeamKey, UpdatedOn, CreatedOn)
SELECT 
    C.RosterID, 
    C.TournamentID, 
    @SFaceembeddings,  -- Placeholder for embedding value from code
    @ImagePath,        -- Placeholder for image path from code
    C.TeamKey, 
    @UpdatedOn,        -- Manual input for UpdatedOn
    @CreatedOn         -- Manual input for CreatedOn
FROM Constellation C"""


In [107]:
query = """
INSERT INTO PlayerPhotoEmbedding (PlayerEmbeddingID, RosterID, TournamentID, SFaceEmbeddings, ImgePath, TeamKey, UpdatedOn, CreatedOn)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""
player_embedding_id = 1
roster_id = 1
tournament_id = 324
s_face_embedding = studio_embedding
image_path = embedding
TeamKey = 12345
UpdatedOn = None
CreatedOn = None

cur.execute(query, (player_embedding_id, roster_id, tournament_id, s_face_embedding, image_path, TeamKey, UpdatedOn, CreatedOn))
cnxn.commit()

ProgrammingError: ("A TVP's rows must all be the same size.", 'HY000')

In [114]:
cnxn.close()

In [ ]:
array_1 = [[-7.85858259e-02  7.12907538e-02  6.14989735e-03  2.01968499e-03
   1.05386630e-01  6.76766112e-02  4.50795703e-02 -9.12087858e-02
   2.40475461e-02  2.26881132e-02  1.99639313e-02 -6.04591565e-03
  -2.26022545e-02 -2.12184414e-02 -3.76298614e-02  1.69947855e-02
   5.76241016e-02  7.03313351e-02  1.89078785e-02 -1.18807203e-03
  -2.36736480e-02 -1.72245875e-02  1.49688914e-01 -2.16134153e-02
  -1.49916857e-02 -1.03599988e-02  8.34512413e-02 -1.15387021e-02
  -3.25125456e-02 -2.35740747e-02 -2.46273447e-02  2.66832728e-02
  -8.75416696e-02  5.00328861e-05  4.18236107e-02  2.25926028e-03
   1.18978530e-01 -9.26278159e-02 -5.84242716e-02 -9.18131135e-03
   3.63990851e-02 -2.71755550e-03  2.47715712e-02  4.30520289e-02
   3.53986718e-04 -4.39844504e-02  4.32560407e-03  9.97797921e-02
  -8.27569216e-02 -6.44123275e-03 -2.87720542e-02  8.26075021e-03
   6.92043919e-03 -2.58962251e-02 -7.29530444e-03 -2.28166860e-02
  -1.05293784e-02  1.76740773e-02  3.45979184e-02  6.28289729e-02
   5.03405482e-02  9.17313271e-04  1.56406965e-02 -3.52854393e-02
  -2.10615303e-02 -3.92226689e-02 -2.82533076e-02  1.94136500e-02
  -1.95740517e-02  9.76680871e-03  7.89739937e-03 -5.20645864e-02
   7.41816908e-02 -1.75762770e-03  3.04241162e-02 -4.79849502e-02
   2.15197969e-02  1.04255313e-02 -8.85084923e-03 -1.49933454e-02
  -6.34391978e-02 -2.80230939e-02 -6.31471798e-02  4.35941759e-03
  -2.92150285e-02 -1.94797677e-03 -1.57842860e-02  2.08167825e-02
  -5.37427962e-02 -1.44901928e-02 -2.87889186e-02  9.67802107e-03
   1.29646827e-02 -2.28332318e-02  1.15639139e-02 -5.20023406e-02
  -2.98444331e-02  4.71804515e-02  5.79011219e-04  6.76530600e-02
   3.88565799e-03  3.47757004e-02 -1.12002485e-01  1.59905385e-02
  -5.71685061e-02  2.19253469e-02 -2.60547772e-02  2.37526242e-02
   3.20684165e-03 -1.61613896e-02  5.93491234e-02 -6.04400821e-02
  -1.59658380e-02  3.64940725e-02  7.59720197e-03  2.79634129e-02
   2.96679838e-03  1.35062905e-02  1.54368067e-02 -2.39694528e-02
   9.17518362e-02 -4.96603642e-03  6.55537099e-02  5.18396944e-02
   1.66625734e-02 -5.73155470e-02 -7.59790139e-03  9.43220183e-02
  -3.22084911e-02  3.74817103e-02 -7.38385692e-02 -4.71956730e-02
   2.27542035e-02 -5.21347523e-02 -6.73806518e-02  5.95778823e-02
  -4.40821759e-02 -6.94389865e-02 -3.56284678e-02  3.04507073e-02
  -9.65606980e-03 -8.59786049e-02 -2.14630198e-02  3.18369195e-02
  -4.47963253e-02 -2.57919542e-02  8.91646743e-03 -2.22809538e-02
   8.37932974e-02 -5.00750914e-02 -1.94424037e-02  7.04114959e-02
   7.87835568e-03  2.59105559e-03  4.87441458e-02 -5.66135608e-02
  -1.19226063e-02  2.71758903e-02 -6.15025964e-03  1.45611512e-02
   5.59288636e-02  8.22071731e-02  3.44940368e-03  4.44333516e-02
   4.69541699e-02 -7.73124350e-03  3.64487153e-03 -1.18693681e-02
  -4.91062514e-02 -2.23374926e-02 -1.12697454e-02  9.49645508e-03
  -2.74382159e-03 -2.32496187e-02  3.32912285e-04 -4.12296690e-02
   2.50900239e-02  3.62673253e-02 -4.49263975e-02  4.14755344e-02
  -3.39121968e-02  1.16720140e-01 -2.23896150e-02 -2.59592291e-02
  -2.19265763e-02 -3.03970221e-02 -4.99693044e-02 -2.24840664e-03
  -8.18728749e-03  2.94108856e-02 -3.68940085e-02  6.83805672e-03
   3.32868174e-02 -5.89892492e-02  5.19727496e-03 -5.16012907e-02
  -2.65075117e-02  8.13418999e-03  2.96069924e-02  1.31653026e-02
   5.62360231e-03 -9.42009836e-02 -2.29766853e-02 -2.42116675e-02
   2.09213840e-03  1.19759804e-02 -3.09383422e-02 -5.50317056e-02
  -4.31498848e-02  3.24803852e-02 -9.54433111e-04 -1.79582201e-02
  -1.12125631e-02 -2.63631828e-02 -3.09319124e-02  6.61286712e-02
   2.73198653e-02 -1.41057223e-02 -1.36660114e-02 -2.83613708e-03
   3.86521257e-02 -3.76581550e-02  9.80198011e-02 -6.49555773e-02
   2.18478329e-02  1.01522148e-01  1.30588179e-02  1.18060103e-02
   1.02808215e-02  5.55616803e-02 -7.71301007e-03  3.63279060e-02
  -8.59640352e-03  7.83512741e-02  3.65371108e-02  2.21386310e-02
  -3.94582003e-02 -9.71395224e-02  1.46435369e-02  3.37347463e-02
  -4.64623310e-02 -6.50994107e-02 -8.24916875e-04 -3.07540931e-02
   7.81192482e-02 -5.36969071e-03 -5.24689592e-02  1.69039685e-02
   6.64833412e-02  3.45852249e-03  2.80485861e-02 -7.19003007e-03
   9.25969519e-03 -6.18824698e-02  3.80047858e-02 -5.45680560e-02
  -6.41162042e-03  3.48481201e-02 -1.90443527e-02  1.76015701e-02
  -2.03382727e-02  4.51109186e-02 -2.02030148e-02  6.00176901e-02
  -3.23803425e-02 -5.05402535e-02 -3.77166681e-02 -7.66030550e-02
   3.79392616e-02  8.20330437e-03  3.78911267e-03 -4.78005793e-04
  -1.30871646e-02 -3.43929194e-02 -2.29659975e-02  1.38488654e-02
  -2.40767505e-02  8.31200778e-02  5.10078743e-02 -1.89717510e-03
   5.63424528e-02  5.76290786e-02  1.19760484e-02  2.91834176e-02
  -2.96422411e-02 -9.29331556e-02  1.55954678e-02 -3.25933611e-03
  -2.39883224e-03 -2.57964879e-02 -1.03805456e-02 -7.19087943e-03
  -5.24081476e-02 -3.63600478e-02  3.08204107e-02  6.10895567e-02
  -3.90643347e-03  7.34980553e-02 -1.25038214e-02 -3.43297794e-02
   2.11460143e-02  7.37809092e-02 -6.76054060e-02  1.10907257e-02
   3.44633907e-02 -8.87159631e-03  3.63588072e-02 -2.84776930e-02
  -6.85571581e-02 -4.28044610e-02  1.68109685e-02 -4.33694534e-02
  -6.88249245e-02 -7.90993124e-02  5.28645366e-02  6.54709861e-02
   3.08322348e-02 -5.69169689e-03  3.40434089e-02 -4.62870747e-02
   8.17391425e-02 -1.35192312e-02  1.17729865e-02  8.90713781e-02
   2.30759587e-02  3.48259024e-02  3.79126519e-02  5.22870794e-02
   3.42694931e-02 -2.52284091e-02 -2.99401861e-02 -3.23126912e-02
   2.98875347e-02  4.77116331e-02 -2.75164406e-04  3.93741950e-03
   1.00288928e-01  3.48616615e-02 -8.19456000e-06 -5.46923047e-03
  -3.59344929e-02 -3.30604538e-02  6.50439085e-03  5.17821051e-02
   8.25907849e-03  5.44187203e-02 -1.35432547e-02  4.87234145e-02
   8.67644232e-03  2.30474677e-02  1.16133643e-03 -4.33820672e-03
   6.71808235e-03  1.34237669e-02  2.51314975e-03 -1.26300508e-03
   2.18236689e-02  3.12099159e-02 -2.34017498e-03  2.82459576e-02
   1.16364509e-01 -6.09342428e-03 -1.06660426e-01 -1.09601662e-01
  -5.84628098e-02  1.59160048e-02 -4.75380383e-02 -3.54579575e-02
  -1.95017811e-02 -4.35459688e-02 -4.10042331e-02  2.86331028e-02
  -1.44052682e-02  7.58064762e-02  5.10720946e-02  6.20800480e-02
  -2.04272512e-02  8.71391036e-03 -5.67162922e-03  3.99318431e-03
  -1.80225857e-02  6.09160922e-02 -5.67296194e-03 -2.19942238e-02
  -1.92127824e-02 -1.16258617e-02 -4.99209687e-02  3.74855287e-02
   4.24178168e-02 -7.90876430e-03 -6.19143201e-03  5.62637709e-02
  -2.53300499e-02  3.62723507e-02 -7.89754838e-03 -6.77008554e-02
   2.67957579e-02  3.68171185e-03  1.16285477e-02 -7.39637762e-03
  -3.41182426e-02 -1.79545749e-02 -5.14444597e-02 -2.73268018e-02
   8.63852259e-03 -2.55110469e-02 -2.73581507e-04 -1.24415262e-02
   1.11916356e-01  7.03705102e-02  1.23385135e-02  5.85803613e-02
  -2.56735105e-02  8.78301859e-02 -1.88815072e-02 -4.59191389e-02
  -5.72144724e-02  2.82341838e-02 -3.60980135e-04 -1.20926006e-02
   4.55328897e-02 -6.38948381e-02 -5.90753481e-02  6.92005968e-03
   4.03027795e-02  5.86436801e-02  3.52707729e-02 -5.33954091e-02
  -1.16133234e-02  5.67868352e-02 -1.22383945e-01  3.05117089e-02
   5.25141954e-02 -3.24655212e-02 -5.27777337e-02 -8.46767649e-02
  -1.25925504e-02 -3.74070108e-02 -5.56155406e-02 -4.91401069e-02
  -2.02490631e-02  5.76147102e-02 -2.56996471e-02  9.17950762e-04
  -5.12101203e-02 -1.28514199e-02 -6.16678596e-02  5.83632430e-03
   5.31936660e-02 -4.67110984e-02  5.99850640e-02 -6.06771209e-04
  -3.39666717e-02 -4.68931906e-02 -3.77025418e-02  2.18738709e-02
  -6.57394007e-02  9.84870940e-02  8.72209519e-02  2.63794828e-02
   4.62754443e-02  6.97921216e-02 -9.03559476e-02  4.71533462e-02
   1.31393466e-02  3.06900293e-02 -2.14917790e-02 -2.74315886e-02
   6.72509968e-02  2.27177069e-02  3.02601196e-02  1.38179986e-02
  -4.04901579e-02 -1.66330766e-02 -5.86083252e-03  5.70054390e-02
   9.60947871e-02  8.36022347e-02 -7.22318664e-02 -2.04544999e-02
   8.14385479e-04  1.54807745e-02  2.91733490e-03  6.73428643e-03
  -3.38655151e-02  8.59013572e-02 -7.10195396e-04  2.36029960e-02
   1.32758673e-02 -8.92666206e-02  2.13320423e-02 -7.62407435e-03
  -1.30890459e-01 -1.21397674e-02  6.65325001e-02  1.74030196e-02
  -9.54813138e-02  2.80318968e-02 -9.47578810e-03  4.44188528e-02
  -8.55924562e-02 -2.53794994e-02  4.01553363e-02  3.96558829e-02
   2.84677483e-02 -2.38389926e-04  5.08345366e-02 -7.63418106e-03
  -3.13599519e-02 -2.98188962e-02  6.59498014e-03 -1.81806851e-02]]

In [ ]:
array_2 = [[-7.85858259e-02  7.12907538e-02  6.14989735e-03  2.01968499e-03
   1.05386630e-01  6.76766112e-02  4.50795703e-02 -9.12087858e-02
   2.40475461e-02  2.26881132e-02  1.99639313e-02 -6.04591565e-03
  -2.26022545e-02 -2.12184414e-02 -3.76298614e-02  1.69947855e-02
   5.76241016e-02  7.03313351e-02  1.89078785e-02 -1.18807203e-03
  -2.36736480e-02 -1.72245875e-02  1.49688914e-01 -2.16134153e-02
  -1.49916857e-02 -1.03599988e-02  8.34512413e-02 -1.15387021e-02
  -3.25125456e-02 -2.35740747e-02 -2.46273447e-02  2.66832728e-02
  -8.75416696e-02  5.00328861e-05  4.18236107e-02  2.25926028e-03
   1.18978530e-01 -9.26278159e-02 -5.84242716e-02 -9.18131135e-03
   3.63990851e-02 -2.71755550e-03  2.47715712e-02  4.30520289e-02
   3.53986718e-04 -4.39844504e-02  4.32560407e-03  9.97797921e-02
  -8.27569216e-02 -6.44123275e-03 -2.87720542e-02  8.26075021e-03
   6.92043919e-03 -2.58962251e-02 -7.29530444e-03 -2.28166860e-02
  -1.05293784e-02  1.76740773e-02  3.45979184e-02  6.28289729e-02
   5.03405482e-02  9.17313271e-04  1.56406965e-02 -3.52854393e-02
  -2.10615303e-02 -3.92226689e-02 -2.82533076e-02  1.94136500e-02
  -1.95740517e-02  9.76680871e-03  7.89739937e-03 -5.20645864e-02
   7.41816908e-02 -1.75762770e-03  3.04241162e-02 -4.79849502e-02
   2.15197969e-02  1.04255313e-02 -8.85084923e-03 -1.49933454e-02
  -6.34391978e-02 -2.80230939e-02 -6.31471798e-02  4.35941759e-03
  -2.92150285e-02 -1.94797677e-03 -1.57842860e-02  2.08167825e-02
  -5.37427962e-02 -1.44901928e-02 -2.87889186e-02  9.67802107e-03
   1.29646827e-02 -2.28332318e-02  1.15639139e-02 -5.20023406e-02
  -2.98444331e-02  4.71804515e-02  5.79011219e-04  6.76530600e-02
   3.88565799e-03  3.47757004e-02 -1.12002485e-01  1.59905385e-02
  -5.71685061e-02  2.19253469e-02 -2.60547772e-02  2.37526242e-02
   3.20684165e-03 -1.61613896e-02  5.93491234e-02 -6.04400821e-02
  -1.59658380e-02  3.64940725e-02  7.59720197e-03  2.79634129e-02
   2.96679838e-03  1.35062905e-02  1.54368067e-02 -2.39694528e-02
   9.17518362e-02 -4.96603642e-03  6.55537099e-02  5.18396944e-02
   1.66625734e-02 -5.73155470e-02 -7.59790139e-03  9.43220183e-02
  -3.22084911e-02  3.74817103e-02 -7.38385692e-02 -4.71956730e-02
   2.27542035e-02 -5.21347523e-02 -6.73806518e-02  5.95778823e-02
  -4.40821759e-02 -6.94389865e-02 -3.56284678e-02  3.04507073e-02
  -9.65606980e-03 -8.59786049e-02 -2.14630198e-02  3.18369195e-02
  -4.47963253e-02 -2.57919542e-02  8.91646743e-03 -2.22809538e-02
   8.37932974e-02 -5.00750914e-02 -1.94424037e-02  7.04114959e-02
   7.87835568e-03  2.59105559e-03  4.87441458e-02 -5.66135608e-02
  -1.19226063e-02  2.71758903e-02 -6.15025964e-03  1.45611512e-02
   5.59288636e-02  8.22071731e-02  3.44940368e-03  4.44333516e-02
   4.69541699e-02 -7.73124350e-03  3.64487153e-03 -1.18693681e-02
  -4.91062514e-02 -2.23374926e-02 -1.12697454e-02  9.49645508e-03
  -2.74382159e-03 -2.32496187e-02  3.32912285e-04 -4.12296690e-02
   2.50900239e-02  3.62673253e-02 -4.49263975e-02  4.14755344e-02
  -3.39121968e-02  1.16720140e-01 -2.23896150e-02 -2.59592291e-02
  -2.19265763e-02 -3.03970221e-02 -4.99693044e-02 -2.24840664e-03
  -8.18728749e-03  2.94108856e-02 -3.68940085e-02  6.83805672e-03
   3.32868174e-02 -5.89892492e-02  5.19727496e-03 -5.16012907e-02
  -2.65075117e-02  8.13418999e-03  2.96069924e-02  1.31653026e-02
   5.62360231e-03 -9.42009836e-02 -2.29766853e-02 -2.42116675e-02
   2.09213840e-03  1.19759804e-02 -3.09383422e-02 -5.50317056e-02
  -4.31498848e-02  3.24803852e-02 -9.54433111e-04 -1.79582201e-02
  -1.12125631e-02 -2.63631828e-02 -3.09319124e-02  6.61286712e-02
   2.73198653e-02 -1.41057223e-02 -1.36660114e-02 -2.83613708e-03
   3.86521257e-02 -3.76581550e-02  9.80198011e-02 -6.49555773e-02
   2.18478329e-02  1.01522148e-01  1.30588179e-02  1.18060103e-02
   1.02808215e-02  5.55616803e-02 -7.71301007e-03  3.63279060e-02
  -8.59640352e-03  7.83512741e-02  3.65371108e-02  2.21386310e-02
  -3.94582003e-02 -9.71395224e-02  1.46435369e-02  3.37347463e-02
  -4.64623310e-02 -6.50994107e-02 -8.24916875e-04 -3.07540931e-02
   7.81192482e-02 -5.36969071e-03 -5.24689592e-02  1.69039685e-02
   6.64833412e-02  3.45852249e-03  2.80485861e-02 -7.19003007e-03
   9.25969519e-03 -6.18824698e-02  3.80047858e-02 -5.45680560e-02
  -6.41162042e-03  3.48481201e-02 -1.90443527e-02  1.76015701e-02
  -2.03382727e-02  4.51109186e-02 -2.02030148e-02  6.00176901e-02
  -3.23803425e-02 -5.05402535e-02 -3.77166681e-02 -7.66030550e-02
   3.79392616e-02  8.20330437e-03  3.78911267e-03 -4.78005793e-04
  -1.30871646e-02 -3.43929194e-02 -2.29659975e-02  1.38488654e-02
  -2.40767505e-02  8.31200778e-02  5.10078743e-02 -1.89717510e-03
   5.63424528e-02  5.76290786e-02  1.19760484e-02  2.91834176e-02
  -2.96422411e-02 -9.29331556e-02  1.55954678e-02 -3.25933611e-03
  -2.39883224e-03 -2.57964879e-02 -1.03805456e-02 -7.19087943e-03
  -5.24081476e-02 -3.63600478e-02  3.08204107e-02  6.10895567e-02
  -3.90643347e-03  7.34980553e-02 -1.25038214e-02 -3.43297794e-02
   2.11460143e-02  7.37809092e-02 -6.76054060e-02  1.10907257e-02
   3.44633907e-02 -8.87159631e-03  3.63588072e-02 -2.84776930e-02
  -6.85571581e-02 -4.28044610e-02  1.68109685e-02 -4.33694534e-02
  -6.88249245e-02 -7.90993124e-02  5.28645366e-02  6.54709861e-02
   3.08322348e-02 -5.69169689e-03  3.40434089e-02 -4.62870747e-02
   8.17391425e-02 -1.35192312e-02  1.17729865e-02  8.90713781e-02
   2.30759587e-02  3.48259024e-02  3.79126519e-02  5.22870794e-02
   3.42694931e-02 -2.52284091e-02 -2.99401861e-02 -3.23126912e-02
   2.98875347e-02  4.77116331e-02 -2.75164406e-04  3.93741950e-03
   1.00288928e-01  3.48616615e-02 -8.19456000e-06 -5.46923047e-03
  -3.59344929e-02 -3.30604538e-02  6.50439085e-03  5.17821051e-02
   8.25907849e-03  5.44187203e-02 -1.35432547e-02  4.87234145e-02
   8.67644232e-03  2.30474677e-02  1.16133643e-03 -4.33820672e-03
   6.71808235e-03  1.34237669e-02  2.51314975e-03 -1.26300508e-03
   2.18236689e-02  3.12099159e-02 -2.34017498e-03  2.82459576e-02
   1.16364509e-01 -6.09342428e-03 -1.06660426e-01 -1.09601662e-01
  -5.84628098e-02  1.59160048e-02 -4.75380383e-02 -3.54579575e-02
  -1.95017811e-02 -4.35459688e-02 -4.10042331e-02  2.86331028e-02
  -1.44052682e-02  7.58064762e-02  5.10720946e-02  6.20800480e-02
  -2.04272512e-02  8.71391036e-03 -5.67162922e-03  3.99318431e-03
  -1.80225857e-02  6.09160922e-02 -5.67296194e-03 -2.19942238e-02
  -1.92127824e-02 -1.16258617e-02 -4.99209687e-02  3.74855287e-02
   4.24178168e-02 -7.90876430e-03 -6.19143201e-03  5.62637709e-02
  -2.53300499e-02  3.62723507e-02 -7.89754838e-03 -6.77008554e-02
   2.67957579e-02  3.68171185e-03  1.16285477e-02 -7.39637762e-03
  -3.41182426e-02 -1.79545749e-02 -5.14444597e-02 -2.73268018e-02
   8.63852259e-03 -2.55110469e-02 -2.73581507e-04 -1.24415262e-02
   1.11916356e-01  7.03705102e-02  1.23385135e-02  5.85803613e-02
  -2.56735105e-02  8.78301859e-02 -1.88815072e-02 -4.59191389e-02
  -5.72144724e-02  2.82341838e-02 -3.60980135e-04 -1.20926006e-02
   4.55328897e-02 -6.38948381e-02 -5.90753481e-02  6.92005968e-03
   4.03027795e-02  5.86436801e-02  3.52707729e-02 -5.33954091e-02
  -1.16133234e-02  5.67868352e-02 -1.22383945e-01  3.05117089e-02
   5.25141954e-02 -3.24655212e-02 -5.27777337e-02 -8.46767649e-02
  -1.25925504e-02 -3.74070108e-02 -5.56155406e-02 -4.91401069e-02
  -2.02490631e-02  5.76147102e-02 -2.56996471e-02  9.17950762e-04
  -5.12101203e-02 -1.28514199e-02 -6.16678596e-02  5.83632430e-03
   5.31936660e-02 -4.67110984e-02  5.99850640e-02 -6.06771209e-04
  -3.39666717e-02 -4.68931906e-02 -3.77025418e-02  2.18738709e-02
  -6.57394007e-02  9.84870940e-02  8.72209519e-02  2.63794828e-02
   4.62754443e-02  6.97921216e-02 -9.03559476e-02  4.71533462e-02
   1.31393466e-02  3.06900293e-02 -2.14917790e-02 -2.74315886e-02
   6.72509968e-02  2.27177069e-02  3.02601196e-02  1.38179986e-02
  -4.04901579e-02 -1.66330766e-02 -5.86083252e-03  5.70054390e-02
   9.60947871e-02  8.36022347e-02 -7.22318664e-02 -2.04544999e-02
   8.14385479e-04  1.54807745e-02  2.91733490e-03  6.73428643e-03
  -3.38655151e-02  8.59013572e-02 -7.10195396e-04  2.36029960e-02
   1.32758673e-02 -8.92666206e-02  2.13320423e-02 -7.62407435e-03
  -1.30890459e-01 -1.21397674e-02  6.65325001e-02  1.74030196e-02
  -9.54813138e-02  2.80318968e-02 -9.47578810e-03  4.44188528e-02
  -8.55924562e-02 -2.53794994e-02  4.01553363e-02  3.96558829e-02
   2.84677483e-02 -2.38389926e-04  5.08345366e-02 -7.63418106e-03
  -3.13599519e-02 -2.98188962e-02  6.59498014e-03 -1.81806851e-02]]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3661350445.py, line 1)

In [30]:
print("value: ", array_1-array_2)

NameError: name 'array_1' is not defined